In [5]:
import pandas as pd
from prophet import Prophet
import plotly
import os

In [6]:
df = pd.read_csv(os.path.join("refined", "top_aerolinea.csv"))

In [10]:
df= df.iloc[:,1:]

In [13]:
df["Fecha"] = pd.to_datetime(df["Fecha"])

In [14]:
df.dtypes

Fecha        datetime64[ns]
pasajeros             int64
dtype: object

In [15]:
m = Prophet()
m.fit(df)

AttributeError: 'Prophet' object has no attribute 'stan_backend'